# Lagrangian velocity gradient
1. Compute a particle trajectory.
2. Get the velocity gradient along the path.

In [1]:
import numpy as np

from pyJHTDB import libJHTDB

lJHTDB = libJHTDB()
lJHTDB.initialize()

#data_name = 'channel' 
data_name = 'isotropic1024coarse'

#Add token
auth_token  = "" #Replace with your own token here
lJHTDB.add_token(auth_token)

# Starting coordinate
p0 = np.array([0., 0., 0.])

Nt = 500

# Restart
new_calc = False

In [ ]:

positions, times = lJHTDB.getPosition(
    starttime = 0.,
    endtime = 25.,
    dt = 0.0065,
    point_coords = p0,
    steps_to_keep = Nt-1,
    data_set = data_name) 

lJHTDB.finalize()
new_calc = True
np.savez("restart", times=times, positions=positions)


starting integration loop, dataset is  isotropic1024coarse
at time step 1 out of 499
got next position for time step 1
at time step 2 out of 499
got next position for time step 2
at time step 3 out of 499
got next position for time step 3
at time step 4 out of 499
got next position for time step 4
at time step 5 out of 499
got next position for time step 5
at time step 6 out of 499
got next position for time step 6
at time step 7 out of 499
got next position for time step 7
at time step 8 out of 499
got next position for time step 8
at time step 9 out of 499
got next position for time step 9
at time step 10 out of 499
got next position for time step 10
at time step 11 out of 499
got next position for time step 11
at time step 12 out of 499
got next position for time step 12
at time step 13 out of 499
got next position for time step 13
at time step 14 out of 499
got next position for time step 14
at time step 15 out of 499
got next position for time step 15
at time step 16 out of 499
go

got next position for time step 132
at time step 133 out of 499
got next position for time step 133
at time step 134 out of 499
got next position for time step 134
at time step 135 out of 499
got next position for time step 135
at time step 136 out of 499
got next position for time step 136
at time step 137 out of 499
got next position for time step 137
at time step 138 out of 499
got next position for time step 138
at time step 139 out of 499
got next position for time step 139
at time step 140 out of 499
got next position for time step 140
at time step 141 out of 499
got next position for time step 141
at time step 142 out of 499
got next position for time step 142
at time step 143 out of 499
got next position for time step 143
at time step 144 out of 499
got next position for time step 144
at time step 145 out of 499
got next position for time step 145
at time step 146 out of 499
got next position for time step 146
at time step 147 out of 499
got next position for time step 147
at t

In [ ]:
if not new_calc:
    data = np.load("restart.npz")
    times, positions = data['times'], data['positions']

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig = plt.figure(figsize = (10, 5))
a = fig.add_subplot(111)
a.plot(positions[:,0, 0], positions[:,0, 1], '.-')
plt.show()

In [ ]:
gradU = []
lJHTDB.initialize()
for t, x in zip(times, positions[:,0,:]):
    # Get the first element (only one point)
    gradU.append(lJHTDB.getData(
                   t,
                   x.reshape((1,3)),
                   sinterp = 44,
                    data_set = data_name,
                   getFunction='getVelocityGradient')[0])  
lJHTDB.finalize()

In [ ]:
gradU = np.array(gradU)
gradU = gradU.reshape((Nt,3,3))
gradU = gradU - np.trace(gradU, axis1=1, axis2=2).reshape((Nt,1,1))/3*np.full((Nt,3,3),np.identity(3))

In [ ]:
gradU = np.transpose(gradU, axes=(0,2,1))
out = np.hstack((times.reshape(Nt, 1), gradU.reshape(Nt, 9)))
out

In [ ]:
np.savez(f"t_gradU_{data_name}_y{p0[1]}", t=times, gradU=gradU.reshape(Nt, 9), p0=p0)

In [ ]:
labels = ['xx','xy','xz','yx','yy','yz','zx','zy','zz']
for i,label in enumerate(labels):
    plt.plot(times, gradU.reshape(Nt, 9)[:,i], label=label)
plt.legend()
plt.show()